In [ ]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project2026.zip
!unzip project2026.zip
!ls -l

!git clone https://github.com/ArturoBenedettiMichelangeli/GO_Neural_Network.git

--2025-10-27 09:13:10--  https://www.lamsade.dauphine.fr/~cazenave/project2026.zip
Resolving www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)... 193.48.71.250
Connecting to www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)|193.48.71.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138578548 (132M) [application/zip]
Saving to: ‘project2026.zip’

project2026.zip     100%[===================>] 132.16M  14.3MB/s    in 11s     

2025-10-27 09:13:22 (12.3 MB/s) - ‘project2026.zip’ saved [138578548/138578548]

Archive:  project2026.zip
  inflating: games.data              
  inflating: golois.cpython-312-x86_64-linux-gnu.so  
total 665408
-rw-r--r-- 1 root root 542497580 Oct  7  2022 games.data
-rwxr-xr-x 1 root root    284672 Oct  1 15:09 golois.cpython-312-x86_64-linux-gnu.so
-rw-r--r-- 1 root root 138578548 Oct  1 20:02 project2026.zip
drwxr-xr-x 1 root root      4096 Oct 23 13:40 sample_data
Cloning into 'GO_Neural_Network'...
remote: Enumerating objects: 34,

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import losses
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
import math
import gc

import golois

tf.keras.backend.clear_session(free_memory=True)


class CustomScheduleCA(LearningRateSchedule):
    """
    Cosine Annealing avec warmup
    - Utilise tf.keras.optimizers.schedules.CosineDecay pour le Decay.
    - Ajoute la logique du Warmup au début.
    """
    def __init__(self, initial_learning_rate, decay_steps, alpha=0.0,
                 name='CosineDecay', warmup_steps=0):
        super(CustomScheduleCA, self).__init__()

        # Définition des attributs
        self.initial_learning_rate = tf.cast(initial_learning_rate, tf.float32)
        self.decay_steps = tf.cast(decay_steps, tf.float32)
        self.alpha = tf.cast(alpha, tf.float32)
        self.warmup_steps = tf.cast(warmup_steps, tf.float32)
        self.name = name

        self.cosine_decay_schedule = tf.keras.optimizers.schedules.CosineDecay(
            initial_learning_rate=self.initial_learning_rate,
            decay_steps=self.decay_steps,
            alpha=self.alpha,
            name=self.name
        )

    def __call__(self, step):
        step = tf.cast(step, tf.float32)

        warmup_lr = self.initial_learning_rate * (step / self.warmup_steps)

        decay_lr = self.cosine_decay_schedule(step - self.warmup_steps)

        lr = tf.cond(
            tf.math.less(step, self.warmup_steps),
            lambda: warmup_lr,
            lambda: decay_lr
        )

        return lr

    def get_config(self):
        return {
            'initial_learning_rate': float(self.initial_learning_rate.numpy()),
            'decay_steps': int(self.decay_steps.numpy()),
            'alpha': float(self.alpha.numpy()),
            'name': self.name,
            'warmup_steps': int(self.warmup_steps.numpy())
        }


planes = 31
moves = 361
N = 10000
epochs = 600
batch = 128
filters = 32

input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype('float32')

policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical(policy)
policy = policy.astype('float32')

value = np.random.randint(2, size=(N,))
value = value.astype('float32')

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype('float32')

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype('float32')

print("Tensorflow version", tf.__version__)
print("getValidation", flush=True)
golois.getValidation(input_data, policy, value, end)


input_layer = keras.Input(shape=(19, 19, planes), name='board')
x = layers.Conv2D(filters, 1, activation='relu', padding='same')(input_layer)

# Blocs résiduels
for i in range(4):
    ident = x
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.add([ident, x])
    x = layers.ReLU()(x)

# Mécanisme d'Attention SE
se = layers.GlobalAveragePooling2D()(x)
se = layers.Dense(filters // 8, activation='relu')(se)
se = layers.Dense(filters, activation='sigmoid')(se)
se = layers.Reshape((1, 1, filters))(se)
x = layers.Multiply()([x, se])

# Policy head
policy_head = layers.Conv2D(1, 1, activation='relu', padding='same',
                            use_bias=False,
                            kernel_regularizer=regularizers.l2(0.0001))(x)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Activation('softmax', name='policy')(policy_head)

# Value head
value_head = layers.Conv2D(1, 1, activation='relu', padding='same',
                          use_bias=False,
                          kernel_regularizer=regularizers.l2(0.0001))(x)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(50, activation='relu',
                         kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value',
                         kernel_regularizer=regularizers.l2(0.0001))(value_head)

model = keras.Model(inputs=input_layer, outputs=[policy_head, value_head])

model.summary()


steps_per_epoch = N // batch
total_steps = steps_per_epoch * epochs

# Configuration du learning rate schedule
initial_lr = 0.001
warmup_epochs = 10
warmup_steps = steps_per_epoch * warmup_epochs
decay_steps = total_steps - warmup_steps
min_lr_ratio = 0.0001

lr_schedule = CustomScheduleCA(
    initial_learning_rate=initial_lr,
    decay_steps=decay_steps,
    alpha=min_lr_ratio,
    warmup_steps=warmup_steps
)

optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(
    optimizer=optimizer,
    loss={
        'policy': losses.CategoricalCrossentropy(),
        'value': keras.losses.MeanSquaredError()
    },
    loss_weights={'policy': 1.0, 'value': 1.0},
    metrics={
        'policy': keras.metrics.CategoricalAccuracy(),
        'value': keras.metrics.MeanAbsoluteError()
    }
)



for i in range(1, epochs + 1):
    print(f'epoch {i}')
    golois.getBatch(input_data, policy, value, end, groups, i * N)

    history = model.fit(
        input_data,
        [policy, value],
        epochs=1,
        batch_size=batch,
        verbose=1
    )

    if i % 5 == 0:
        gc.collect()

    if i % 100 == 0:
        golois.getValidation(input_data, policy, value, end)
        val = model.evaluate(
            input_data,
            [policy, value],
            verbose=2,
            batch_size=batch
        )
        print(f"val = {val}")
        model.save('/content/ChannelAttention_following_ResNet_CosineAnnealing.h5')


        model.summary()



Tensorflow version 2.19.0
getValidation


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ board (InputLayer)  │ (None, 19, 19,    │          0 │ -                 │
│                     │ 31)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 19, 19,    │      1,024 │ board[0][0]       │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 19, 19,    │      9,248 │ conv2d[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 19, 19,    │        128 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 19, 19,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 19, 19,    │      9,248 │ re_lu[0][0]       │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        128 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 19, 19,    │          0 │ conv2d[0][0],     │
│                     │ 32)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 19, 19,    │          0 │ add[0][0]         │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 19, 19,    │      9,248 │ re_lu_1[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        128 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 19, 19,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 19, 19,    │      9,248 │ re_lu_2[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        128 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 19, 19,    │          0 │ re_lu_1[0][0],    │
│                     │ 32)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 19, 19,    │          0 │ add_1[0][0]       │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 19, 19,    │      9,248 │ re_lu_3[0][0]   

 Total params: 94,539 (369.29 KB)

 Trainable params: 94,027 (367.29 KB)

...

 Total params: 282,594 (1.08 MB)

 Trainable params: 94,027 (367.29 KB)

 Non-trainable params: 512 (2.00 KB)

 Optimizer params: 188,055 (734.59 KB)


✅ Entraînement terminé avec Cosine Annealing !
